Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name below:

In [29]:
let name = "Kakunuri Shashank Reddy"
let rollno = "CS21B038"

val name : string = "Kakunuri Shashank Reddy"


val rollno : string = "CS21B038"


## Important notes about grading:

1. All code you submit must compile. Programs that do not compile will probably receive an automatic zero. If you are having trouble getting your assignment to compile, please contact the TAs or the instructor or visit the course contact hour. If you run out of time, it is better to comment out the parts that do not compile, than hand in a more complete file that does not compile.
2. All assignments handed in after the deadline will be considered late, and will consume your grace days. 
3. Verify on moodle that you have submitted the correct version, before the deadline. Submitting the incorrect version before the deadline and realizing that you have done so after the deadline will be counted as a late submission.

# Mutability and Modules

In this assignment, you will design and implement a couple of mutable data structures and operations on them.

## Problem 1

Implement structures `IntShowable` and `FloatShowable` that satisfy the following signature. Use `string_of_int: int -> string` and `string_of_float: float -> string` for the corresponding `string_of_t` functions.

In [30]:
module type Showable = sig
  type t
  val string_of_t : t -> string
end

module type Showable = sig type t val string_of_t : t -> string end


In [31]:
module IntShowable :  Showable with type t = int = struct
    type t = int
    let string_of_t t = string_of_int t
end
module FloatShowable : Showable with type t = float = struct
    type t = float
    let string_of_t t = string_of_float t
end

module IntShowable : sig type t = int val string_of_t : t -> string end


module FloatShowable : sig type t = float val string_of_t : t -> string end


In [32]:
(* 5 points *)
assert (IntShowable.string_of_t 10 = "10");
assert (FloatShowable.string_of_t 0.0 = "0.")

- : unit = ()


## Problem 2

Implement a functor 

```ocaml
module MakeNode : 
  functor (C : Showable) -> 
    DoublyLinkedListNode with type content = C.t
``` 

where `DoublyLinkedListNode` is the module type:

In [33]:
module type DoublyLinkedListNode = sig
  type t
  (** The type of doubly linked list node *)
  
  type content
  (** The type of content stored in the doubly linked list node *)

  val create : content -> t 
  (** create a new doubly linked list node with [content] as the content. 
      The next and previous nodes are [None]. *)
  
  val get_next : t -> t option
  (** [get_next t] returns [Some t'] if [t'] is the successor node of [t]. 
      If [t] has no successor, then return [None] *)
      
  val get_prev : t -> t option
  (** [get_prev t] returns [Some t'] if [t'] is the predecessor node of [t]. 
      If [t] has no predecessor, then return [None] *)
      
  val get_content : t -> content
  (** [content t] returns the content [c] of node [t] *)
      
  val set_next : t -> t option -> unit
  (** [set_next t t'] updates the next node of [t] to be [t'] *)
  
  val set_prev : t -> t option -> unit
  (** [set_prev t t'] updates the prev node of [t] to be [t'] *)
  
  val string_of_content : t -> string
  (** [string_of_content t] returns the string form of content stored in the node t *)
end

module type DoublyLinkedListNode =
  sig
    type t
    type content
    val create : content -> t
    val get_next : t -> t option
    val get_prev : t -> t option
    val get_content : t -> content
    val set_next : t -> t option -> unit
    val set_prev : t -> t option -> unit
    val string_of_content : t -> string
  end


In [34]:
(* Implement the functor MakeNode *)
module MakeNode (C : Showable) : DoublyLinkedListNode with type content = C.t = struct
type t = 
{
    mutable next : t option;
    mutable prev : t option;
    v : C.t
}
type content = C.t
let create x = {prev = None; next = None; v = x}
let get_next node = node.next
let get_prev node = node.prev
let get_content node = node.v
let set_next node1 node2 = node1.next <- node2
let set_prev node1 node2 = node1.prev <- node2
let string_of_content node = C.string_of_t node.v
end

module MakeNode :
  functor (C : Showable) ->
    sig
      type t
      type content = C.t
      val create : content -> t
      val get_next : t -> t option
      val get_prev : t -> t option
      val get_content : t -> content
      val set_next : t -> t option -> unit
      val set_prev : t -> t option -> unit
      val string_of_content : t -> string
    end


In [35]:
module IntNode = MakeNode(IntShowable)

module IntNode :
  sig
    type t = MakeNode(IntShowable).t
    type content = IntShowable.t
    val create : content -> t
    val get_next : t -> t option
    val get_prev : t -> t option
    val get_content : t -> content
    val set_next : t -> t option -> unit
    val set_prev : t -> t option -> unit
    val string_of_content : t -> string
  end


In [36]:
(* 5 points *)
let open IntNode in 
let i1 = create 1 in
assert (get_content i1 = 1);

- : unit = ()


In [37]:
(* 5 points *)
let open IntNode in 
let i1,i2,i3 = create 1, create 2, create 3 in
set_next i1 @@ Some i2;
set_next i2 @@ Some i3;
set_prev i2 @@ Some i1;
set_prev i3 @@ Some i2;
let _ = assert (match get_next i1 with None -> false | Some i2 -> get_content i2 = 2) in
()

- : unit = ()


## Problem 3

Implement a functor 

```ocaml 
module MakeList : 
  functor (N : DoublyLinkedListNode) -> 
    DoublyLinkedList with type node = N.t 
                      and type content = N.content
```

where the module type `DoublyLinkedList` is defined as follows:


In [38]:
module type DoublyLinkedList = sig
  type t 
  (** The type of doubly linked list *)
  
  type node
  (** The type of a doubly linked list node *)
  
  type content
  (** The type of content stored in the [node] of doubly linked list *)
    
  val create : unit -> t 
  (** Creates a new (empty) doubly linked list *)
  
  val assign : t -> node option -> unit
  (** [assign t n] makes the node as the head node of the list. 
      The original contents of the list are dropped. *)
  
  val t_of_list : content list -> t
  (** [t_of_list l] returns a new doubly linked list with the list of 
      elements from the list [l]. *)
  
  val is_empty : t -> bool
  (** [is_empty l] return true if [t] is empty *)
  
  val first : t -> node option
  (** [first l] return [Some n] if the list is non-empty. Otherwise, return [None] *)
  
  val insert_first : t -> content -> node
  (** [insert_first l c] inserts a new doubly linked list node [n] as the first 
      node in [l] with [c] as the content. Returns [n]. *)
      
  val insert_after : node -> content -> node
  (** [insert_after n c] inserts a new node [n'] with content [c] after the node [n].
      Returns [n']. *)
  
  val remove : t -> node -> unit
  (** [remove l n] removes the node [n] from the list [l] *)
  
  val iter : t -> (content -> unit) -> unit
  (** [iter l f] applies [f] to each element of the list in the list order. *)
  
  val iter_node : t -> (node -> unit) -> unit
  (** [iter_node l f] applies [f] to each node of the list in the list order. 
      [f] may delete the current node or insert a new node after the current node. 
      In either case, [iter_node] is applied to the rest of the original list. *)
      
  val string_of_t : t -> string
  (** [string_of_t (t_of_list [1;2;3])] returns the string "[1->2->3]".
      [string_of_t (t_of_list [])] returns the string "[]" *)
end

module type DoublyLinkedList =
  sig
    type t
    type node
    type content
    val create : unit -> t
    val assign : t -> node option -> unit
    val t_of_list : content list -> t
    val is_empty : t -> bool
    val first : t -> node option
    val insert_first : t -> content -> node
    val insert_after : node -> content -> node
    val remove : t -> node -> unit
    val iter : t -> (content -> unit) -> unit
    val iter_node : t -> (node -> unit) -> unit
    val string_of_t : t -> string
  end


In [39]:
module MakeList (N : DoublyLinkedListNode) : DoublyLinkedList with type content = N.content 
                                                               and type node = N.t = 
struct

type t = 
{
    mutable head : N.t option
}

type node = N.t

type content = N.content

let create () = {head = None}

let assign t n = t.head <- n

let first node = (node.head)

let rec t_of_list l = 
    match l with
    | [] -> create ()
    | hd :: tl -> 
        let new_node = N.create hd in 
            let rest = t_of_list tl in
                N.set_next new_node (first rest);
                (match first rest with
                 | Some node -> N.set_prev node (Some new_node)
                 | None -> ());
                 assign rest (Some new_node);
                 rest
                 
let is_empty l = (l.head = None)

let insert_first old_list v =
    let new_node = N.create v in 
     match old_list.head with
     | Some old_head ->
       N.set_next new_node (Some old_head);
       N.set_prev old_head (Some new_node);
       assign old_list (Some new_node);
       new_node
    | None ->
        assign old_list (Some new_node);
        new_node

let insert_after n c = 
    let new_node = N.create c in
        match (N.get_next n) with 
        | Some node_next -> 
            N.set_next n (Some new_node);
            N.set_prev new_node (Some n);
            N.set_next new_node (Some node_next);
            N.set_prev node_next (Some new_node);
            new_node
        | None ->
            N.set_next n (Some new_node);
            N.set_prev new_node (Some n);
            new_node
            
let remove l n = 
    match (N.get_prev n) with
    | Some node -> (match (N.get_next n) with 
                   | Some node1 -> (N.set_next node (Some node1)); 
                                   (N.set_prev node1 (Some node))
                   | None -> (N.set_next node None))
                               
    | None -> assign l (N.get_next n) 

let rec iterate f cur = 
    match cur with
    | Some n -> 
        f (N.get_content n);
        iterate f (N.get_next n)
    | None -> ()
let iter l f = iterate f l.head

let rec iterate_node f cur = 
    match cur with
    | Some n -> 
        let next_node = N.get_next n in
        f n;
        iterate_node f next_node
    | None -> ()
        
let iter_node l f = iterate_node f l.head

    
let rec list_to_string cur acc = 
    match cur with
    | Some n -> let content_str = N.string_of_content n in 
                let seperator = match acc with 
                | "" -> "" 
                | _ -> "->" in
                    list_to_string (N.get_next n) (acc ^ seperator ^ content_str)
    | None -> "[" ^ acc ^ "]"
    
let string_of_t l = list_to_string l.head ""
    
end

module MakeList :
  functor (N : DoublyLinkedListNode) ->
    sig
      type t
      type node = N.t
      type content = N.content
      val create : unit -> t
      val assign : t -> node option -> unit
      val t_of_list : content list -> t
      val is_empty : t -> bool
      val first : t -> node option
      val insert_first : t -> content -> node
      val insert_after : node -> content -> node
      val remove : t -> node -> unit
      val iter : t -> (content -> unit) -> unit
      val iter_node : t -> (node -> unit) -> unit
      val string_of_t : t -> string
    end


In [40]:
(* 5 points *)
let module L = MakeList(IntNode) in

let open L in
assert (is_empty (t_of_list []));
let l = t_of_list [1;2;3] in
assert (not @@ is_empty l);
let n = match first l with 
| None -> failwith "impossible" 
| Some n -> n
in
assert (1 = IntNode.get_content n);
L.(assign l (first (t_of_list [4;5;6])));
match first l with
| None -> failwith "impossible"
| Some n -> assert (IntNode.get_content n = 4)


- : unit = ()


In [41]:
(* 5 points *)
let module L = MakeList(IntNode) in

let open L in
let l = t_of_list [1;2;3] in
let n1 = match first l with
  | Some n1 -> n1
  | None -> failwith "impossible"
in
let n2 = match IntNode.get_next n1 with
  | Some n2 -> n2
  | None -> failwith "impossible"
in
remove l n2;
let n3 = match IntNode.get_next n1 with
  | Some n3 -> n3
  | None -> failwith "impossible"
in
let _ = assert (IntNode.get_content n3 = 3) in
remove l n1;
match first l with
| None -> failwith "impossible"
| Some n3 -> assert (IntNode.get_content n3 = 3)


- : unit = ()


In [42]:
(* 10 points *)
let module L = MakeList(IntNode) in
let open L in
let l = t_of_list [1;2;3] in
let s = ref 0 in
iter l (fun c -> s := !s + c);
assert (!s = 6);
iter_node l (fun n -> if IntNode.get_content n mod 2 == 0 then remove l n);
let n1 = match first l with
  | None -> failwith "impossible"
  | Some n -> n
in
assert (IntNode.get_content n1 = 1)


- : unit = ()


## Problem 4

Implement the functor

```ocaml
module MakeDllFunctions :
  functor (N : DoublyLinkedListNode) ->
    Dll_functions with type node = N.t 
                   and type content = N.content 
                   and type dll = MakeList(N).t
```

where the module type `Dll_functions` is:

In [43]:
module type Dll_functions = sig
  type dll
  (** The type of doubly linked list *)
  
  type node
  (** The type of doubly linked list node *)
  
  type content
  (** The type of doubly linked list content *)

  val length : dll -> int
  (** Returns the length of the list. 
      Use [DLL.iter] to implement it.
      Penalty of -5 points DLL.iter is not used *)
  
  val duplicate : dll -> unit
  (** Given a doubly linked list [l] = [1->2], [duplicate l] returns [1->1->2->2]. 
      Use [DLL.iter_node] to implement it.
      Penalty of -5 points DLL.iter_node is not used *)
        
  val rotate : dll -> int -> unit
  (** [rotate l n] rotates the list by [n] nodes. 
      If the list is [1->2->3->4->5], then [rotate l 0] will not modify the list. 
      [rotate l 2] will modify the list to be [3->4->5->1->2]. 
      Assume that [0 <= n < length l] (you can generate exception if the input violates this). *)
  
   val reverse : dll -> unit
   (** [reverse l] reverses the list *)
end

module type Dll_functions =
  sig
    type dll
    type node
    type content
    val length : dll -> int
    val duplicate : dll -> unit
    val rotate : dll -> int -> unit
    val reverse : dll -> unit
  end


In [44]:
module MakeDllFunctions (N : DoublyLinkedListNode) 
  : Dll_functions with type node = N.t 
                   and type content = N.content 
                   and type dll = MakeList(N).t = struct
                   
module DLL = MakeList(N)

type dll = DLL.t

type node = N.t

type content = N.content

let length l =
    let count = ref 0 in
    DLL.iter l (fun _ -> count := !count + 1); 
    !count

    
let duplicate l = 
    DLL.iter_node l (fun node -> 
                    let c = (N.get_content node) in 
                    let copy = c in 
                    DLL.insert_after node copy |> ignore;)


let rec get_last_node cur prev = 
    match cur with
    | Some n -> get_last_node (N.get_next n) cur
    | None -> prev
    
let rotate l n =
  let len = length l in
  if n = 0 || len <= 1 || n >= len then ()
  else (
    let first = DLL.first l in
    let rec find_nth cur count =
      if count = n then cur
      else
        (match cur with
        | Some node -> find_nth (N.get_next node) (count + 1)
        | None -> None)
    in
    let new_first = find_nth first 0 in
    (match new_first with
    | Some new_first_node ->
      let new_last = N.get_prev new_first_node in
      (match new_last with 
      | Some new_last_node -> 
          let last = get_last_node first None in 
          (match last with
          | Some last_node -> 
          (
              (match first with 
              | Some first_node -> 
              (
                  N.set_next new_last_node None;
                  N.set_prev new_first_node None;
                  N.set_prev first_node last;
                  N.set_next last_node first;
                  DLL.assign l new_first
              )
              | None -> ())
          )
          | None -> ())
      | None -> ())
    | None -> ())
  )

                    
let rec reverse_helper l cur prev = 
    match cur with
    | Some n -> 
        let n_next = N.get_next n in 
        N.set_next n prev;
        N.set_prev n n_next;
        reverse_helper l n_next (Some n)
    | None -> DLL.assign l prev
    
let reverse l = reverse_helper l (DLL.first l) None
    
    
end

module MakeDllFunctions :
  functor (N : DoublyLinkedListNode) ->
    sig
      type dll = MakeList(N).t
      type node = N.t
      type content = N.content
      val length : dll -> int
      val duplicate : dll -> unit
      val rotate : dll -> int -> unit
      val reverse : dll -> unit
    end


In [45]:
(* 10 points *)
let module F = MakeDllFunctions (IntNode) in
let module M = MakeList(IntNode) in
let open M in 
let l = t_of_list [1;2;3] in
let _ = assert (F.length l = 3) in
F.duplicate l;
let _ = assert (F.length l = 6) in
()

- : unit = ()


In [46]:
(* 10 points *)
let module F = MakeDllFunctions (IntNode) in
let module M = MakeList(IntNode) in
let open M in 
let l = t_of_list [1;2;3] in
let _ = assert (F.length l = 3) in
F.duplicate l;
let _ = assert (F.length l = 6) in
let _ = assert ("[1->1->2->2->3->3]" = string_of_t l) in
F.rotate l 3;
let _ = assert ("[2->3->3->1->1->2]" = string_of_t l) in
F.rotate l 0;
let _ = assert ("[2->3->3->1->1->2]" = string_of_t l) in
let l = t_of_list [] in
F.rotate l 0;
let _ = assert ("[]" = string_of_t l) in
()

- : unit = ()


In [47]:
(* 10 points *)
let module F = MakeDllFunctions (IntNode) in
let module M = MakeList(IntNode) in
let open M in 
let l = t_of_list [1;2;3] in
F.reverse l;
assert ("[3->2->1]" = string_of_t l)


- : unit = ()


## Hash Table

Implement a hash table using [separate chaining](https://en.wikipedia.org/wiki/Hash_table#Separate_chaining_with_linked_lists). Use an OCaml array for the array of buckets. The documentation of OCaml arrays is found in the [OCaml manual](https://v2.ocaml.org/api/Array.html).

Each element in the array is a doubly linked list in order to allow chaining. Use your doubly linked list that you have implemented earlier. The hash table will be implemented as a functor that accepts two module arguments. The first one defines the type of the key, size of the array and the hash function. The signature of that module is:

In [48]:
module type Key = sig
  type t
  (** The type of key *)
  
  val num_buckets : int
  (** The number of buckets i.e, the size of the buckets array *)
  
  val hash : t -> int
  (** Hashes the key to an integer between [0, num_buckets) *)
  
  val string_of_t : t -> string
  (** Returns the string representation of key *)
end

module type Key =
  sig
    type t
    val num_buckets : int
    val hash : t -> int
    val string_of_t : t -> string
  end


The second functor argument defines the type of content and its signature is `Showable` that we had defined at the beginning of this assignment. 

The signature of the `Hash_table` module is:

In [49]:
module type Hash_table = sig
  type key
  (** The type of key *)
  
  type content
  (** The type of value *)
  
  type t
  (** The type of the hash table *)
  
  val create : unit -> t
  (** Creates a new hash table *)
  
  val put :  t -> key -> content -> unit
  (** [put h k v] associates the key [k] with value [v] in the hash table [h]. 
      If the binding for [k] already exists, overwrite it. *)

  val get : t -> key -> content option
  (** [get h k] returns [Some v], where [v] is the value associated with the 
      key [k] in the hash table [h]. Returns [None] if the [k] is not bound in [h]. *)
      
  val remove : t -> key -> unit
  (** [remove h k] removes the association for key [k] from the hash table [h] if it exists *)
  
  val length : t -> int
  (** Return the number of key-value pairs in the hash table *)
  
  val string_of_t : t -> string
  (** Returns a string version of the hash table. Can be in any format. Only for debugging. *)
  
  val chain_length : t -> int -> int
  (** [chain_length h bucket] returns the length of the doubly linked list 
      at the index [bucket] in the underlying array. 
      Assume that [0 <= bucket < length(bucket_array)] *)
end

module type Hash_table =
  sig
    type key
    type content
    type t
    val create : unit -> t
    val put : t -> key -> content -> unit
    val get : t -> key -> content option
    val remove : t -> key -> unit
    val length : t -> int
    val string_of_t : t -> string
    val chain_length : t -> int -> int
  end


## Problem 5

Implement the functor 

```ocaml
module Make_hash_table :
  functor (K : Key) (C : Showable) ->
    Hash_table with type key = K.t 
                and type content = C.t
```

In [50]:
module Make_hash_table (K : Key) (C : Showable) : Hash_table with type key = K.t and type content = C.t = struct
(* I got 0/35 for this problem :( for not using the double linked list implemented above *)
type key = K.t
type content = C.t

type node = 
{
    mutable key : K.t;
    mutable content : C.t;
    mutable prev : node option;
    mutable next : node option;
}

type t = 
{
    mutable buckets : node option array;
}

let create () =
  let new_buckets = Array.make K.num_buckets None in
  { buckets = new_buckets }


let rec find_key l k = 
    match l with 
    | Some cur -> 
        if cur.key = k then (Some cur)
        else find_key cur.next k
    | None -> None 
    
let rec find_last l prev = 
    match l with 
    | Some cur -> find_last cur.next (Some cur)
    | None -> prev
    
let put h k v = 
    let bucket_index = K.hash k in
    let new_node = {key = k; content = v; prev = None; next = None} in
    let old_node = h.buckets.(bucket_index) in
    let key_dll = find_key old_node k in
    (match key_dll with 
    | Some n -> n.content <- v
    | None -> let last_node = find_last old_node None in 
                (match last_node with
                | Some last_n -> 
                    last_n.next <- (Some new_node);
                    new_node.prev <- last_n.next
                | None -> 
                    h.buckets.(bucket_index) <- (Some new_node)))

let get h k = 
    let bucket_index = K.hash k in 
    let old_node = h.buckets.(bucket_index) in
    let key_dll = find_key old_node k in
    (match key_dll with 
    | Some n -> Some n.content 
    | None -> None)
    
let remove h k = 
    let bucket_index = K.hash k in 
    let rec remove_key_from_dll node = 
        match node with
        | Some n -> 
            if n.key = k then (
            match n.prev with 
            | Some prev_node -> prev_node.next <- n.next
            | None -> h.buckets.(bucket_index) <- n.next
            )
            else remove_key_from_dll n.next
        | None -> ()
    in 
    remove_key_from_dll h.buckets.(bucket_index)

let rec find_length n cur = 
    match n with 
    | Some nd -> find_length nd.next (cur + 1)
    | None -> cur
    
let chain_length h bucket = (find_length h.buckets.(bucket) 0)

let rec length_helper h acc idx = 
    if idx = K.num_buckets then acc
    else length_helper h (acc + (chain_length h idx)) (idx + 1)
    
let length h = length_helper h 0 0

let string_of_t h = ""
        
end

module Make_hash_table :
  functor (K : Key) (C : Showable) ->
    sig
      type key = K.t
      type content = C.t
      type t
      val create : unit -> t
      val put : t -> key -> content -> unit
      val get : t -> key -> content option
      val remove : t -> key -> unit
      val length : t -> int
      val string_of_t : t -> string
      val chain_length : t -> int -> int
    end


In [51]:
#show_module_type Key

module type Key =


  sig
    type t
    val num_buckets : int
    val hash : t -> int
    val string_of_t : t -> string
  end


In [52]:
module K : Key with type t = int = struct
  type t = int
  let num_buckets = 64
  let hash k = k mod num_buckets
  let string_of_t = string_of_int
end

module StringShowable : Showable with type t = string = struct
  type t = string
  let string_of_t s = s
end

module H = Make_hash_table(K)(StringShowable)

open H

module K :
  sig
    type t = int
    val num_buckets : int
    val hash : t -> int
    val string_of_t : t -> string
  end


module StringShowable : sig type t = string val string_of_t : t -> string end


module H :
  sig
    type key = K.t
    type content = StringShowable.t
    type t = Make_hash_table(K)(StringShowable).t
    val create : unit -> t
    val put : t -> key -> content -> unit
    val get : t -> key -> content option
    val remove : t -> key -> unit
    val length : t -> int
    val string_of_t : t -> string
    val chain_length : t -> int -> int
  end


In [53]:
(* 15 points *)
let h = create () in
put h 0 "zero";
put h 1 "one";
assert (get h 0 = Some "zero");
assert (get h 1 = Some "one");
put h 0 "-zero-";
assert (get h 0 = Some "-zero-")


- : unit = ()


In [54]:
(* 20 points *)
let h = create () in
put h 0 "zero";
put h 1 "one";
assert (get h 0 = Some "zero");
assert (get h 1 = Some "one");
put h 64 "-zero-";
assert (get h 0 = Some "zero");
assert (get h 64 = Some "-zero-");
assert (length h = 3);
assert (chain_length h 0 = 2);
assert (chain_length h 1 = 1)


- : unit = ()
